In [ ]:
import os
import random
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset

BASE_DIR = r""
DATASETS = ["2k", "4k", "8k", "16k", "22k"]
NUM_CLASSES = 10
TRAIN_SAMPLES = 100
TEST_SAMPLES  = 100
BATCH_SIZE    = 1
LEARNING_RATE = 1e-3
DEVICE        = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
print(DEVICE)

cuda


In [3]:
class SpectrogramDataset(Dataset):
    def __init__(self, file_list):
        self.file_list = file_list

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        path = self.file_list[idx]
        class_num = int(path.split("/")[-1].split("_")[0])
        spec = np.load(path)
        spec = torch.from_numpy(spec).float()
        spec = spec.unsqueeze(0)
        return spec, class_num

In [4]:
class CNN(nn.Module):
    def __init__(self, num_classes=NUM_CLASSES, model_scaler = 1):
        input_channels = 1
        first_channels = 16*model_scaler
        second_channels = 24*model_scaler
        third_channels = 32*model_scaler
        lin_1 = 128*model_scaler
        lin_2 = 64*model_scaler
        inter_dim = 8*model_scaler
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(input_channels, first_channels, kernel_size=8, padding=1),
            nn.ReLU(inplace=True),
            # nn.MaxPool2d(2),# reduce height and width by 2

            nn.Conv2d(first_channels, second_channels, kernel_size=4, padding=1),
            nn.ReLU(inplace=True),
            # nn.MaxPool2d(2),#reduce height and width by 2

            nn.Conv2d(second_channels, third_channels, kernel_size=4, padding=1),
            nn.ReLU(inplace=True),
            # nn.MaxPool2d(2),#reduce height and width by 2
        )
        
        self.global_pool = nn.AdaptiveAvgPool2d((inter_dim, inter_dim))

        # classifier
        # after pooling batch, 32, 1, 1
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(inter_dim * inter_dim * third_channels, lin_1),
            nn.ReLU(inplace=True),
            nn.Linear(lin_1, lin_2),
            nn.ReLU(inplace=True),
            nn.Linear(lin_2, num_classes)
        )

    def forward(self, x):
        x = self.features(x)       # batch B, 32, H/4, W/4
        x = self.global_pool(x)    # batch, 32, 1, 1
        x = self.classifier(x)     # batch, num_classes
        return x

In [5]:
def train_one_epoch(model, loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    for inputs, targets in tqdm(loader):
        inputs, targets = inputs.to(DEVICE, non_blocking=True), targets.to(DEVICE, non_blocking=True)
        optimizer.zero_grad()
        outputs = model.forward(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    return running_loss / len(loader.dataset)

In [9]:
import time

def evaluate(model, loader):
    with torch.no_grad():
        model.eval()
        correct = 0
        total = 0
        for inputs, targets in loader:
            inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
            outputs = model(inputs)
            preds = outputs.argmax(dim=1)
            correct += (preds == targets).sum().item()
            total += targets.size(0)
        return correct / total

def run_for_dataset(name, skip = False, EPOCHS = 10, model_scaler=1):
    folder = os.path.join(BASE_DIR, name)
    for root, dirs, files in os.walk(folder, topdown=True):
        all_files = [folder + "/" + file for file in files]
    random.shuffle(all_files)
    assert len(all_files) >= TRAIN_SAMPLES + TEST_SAMPLES, \
        f"Not enough samples in {folder}"

    train_files = all_files[:TRAIN_SAMPLES]
    test_files  = all_files[TRAIN_SAMPLES:TRAIN_SAMPLES + TEST_SAMPLES]
    print("Getting Datasets")
    train_ds = SpectrogramDataset(train_files)
    test_ds  = SpectrogramDataset(test_files)
    print("Making Dataloaders")
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,  num_workers=0)
    test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

    model = CNN(model_scaler=model_scaler).to(DEVICE)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    print(f"Training on {name}: {TRAIN_SAMPLES} samples")
    epoch_acc = []
    train_times = []
    test_times = []
    for epoch in range(1, EPOCHS+1):
        time1 = time.time()
        loss = train_one_epoch(model, train_loader, criterion, optimizer)
        time2 = time.time()
        acc  = evaluate(model, test_loader)
        time3 = time.time()
        train_time = time2 - time1
        test_time = time3 - time2
        epoch_acc.append(acc)
        print(f"Epoch {epoch:2d} | loss: {loss:.4f} | test acc: {acc*100:5.2f}%")
        train_times.append(train_time)
        test_times.append(test_time)

    acc = evaluate(model, test_loader)
    print(f"Final test accuracy on {name}: {acc*100:.2f}%")
    return epoch_acc, train_times, test_times
    # out_path = f"{name}_cnn_model.pth"
    # torch.save(model.state_dict(), out_path)
    # print(f"Saved model weights to {out_path}")

In [10]:
if __name__ == "__main__":
    # model_accs = []
    model_train = []
    model_test = []
    for model_scale in [1, 2, 3, 4]:
        print(f"Model size {model_scale}")
        epoch_accs = []
        epoch_trains = []
        epoch_tests = []
        for ds in DATASETS:
            print(f"Dataset {ds}")
            epoch_acc, epoch_train, epoch_test = run_for_dataset(ds, EPOCHS=10, model_scaler=model_scale)
            epoch_accs.append(epoch_acc)
            epoch_trains.append(epoch_train)
            epoch_tests.append(epoch_test)
        # model_accs.append(epoch_accs)
        model_train.append(epoch_trains)
        model_test.append(epoch_tests)

Model size 1
Dataset 2k
Getting Datasets
Making Dataloaders
Training on 2k: 100 samples


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:02<00:00, 46.13it/s]


Epoch  1 | loss: 2.3099 | test acc: 18.00%


100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 104.61it/s]


Epoch  2 | loss: 2.2839 | test acc: 18.00%


...

Epoch  8 | loss: 2.2042 | test acc: 15.00%


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:05<00:00, 19.41it/s]


Epoch  9 | loss: 2.2000 | test acc: 15.00%


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:04<00:00, 22.00it/s]


Epoch 10 | loss: 2.2026 | test acc: 15.00%
Final test accuracy on 22k: 15.00%


In [12]:
print(np.array(model_train).shape)
print(np.array(model_test).shape)

with open("Model_train_data.npy", 'wb') as f:
    np.save(f, np.array(model_train))
with open("Model_inference_data.npy", 'wb') as f:
    np.save(f, np.array(model_test))

(4, 5, 10)
(4, 5, 10)


In [ ]:
# print(np.array(model_accs).shape)
# with open("Model_size_data.npy", 'wb') as f:
#     np.save(f, np.array(model_accs))

In [ ]:
print(model_accs)
model_save = model_accs

In [ ]:
model_save_2 = model_save

In [ ]:
print(len(model_save))
print(len(model_save[3]))
print(len(model_save[2]))
list_of_zeroes = [0 for i in range(10)]
model_save[3].append(list_of_zeroes)
print(len(model_save[3][3]))

In [ ]:
# print(np.array(model_save).shape)
# with open("Model_size_data.npy", 'wb') as f:
#     np.save(f, np.array(model_accs))